##### Copyright 2020 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow Recommenders: Quickstart

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/recommenders/quickstart"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/recommenders/blob/main/docs/examples/quickstart.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/recommenders/blob/main/docs/examples/quickstart.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/recommenders/docs/examples/quickstart.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

In this tutorial, we build a simple matrix factorization model using the [MovieLens 100K dataset](https://grouplens.org/datasets/movielens/100k/) with TFRS. We can use this model to recommend movies for a given user.

### Import TFRS

First, install and import TFRS:

In [1]:
!pip install tensorflow
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

In [2]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf
import pandas as pd
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

### Read the data

In [3]:
# Ratings data.
#ratings = tfds.load('movielens/100k-ratings', split="train")
# Features of all the available movies.
#movies = tfds.load('movielens/100k-movies', split="train")

#ratings

#inputs = [0,1]
#outputs = [0,1]
#data = tf.data.Dataset.from_tensor_slices((inputs,outputs))
#data
    
data = pd.read_csv('data/rating_final_tfds.csv')
data = np.asarray(data).astype('str')
data2 = tf.data.Dataset.from_tensor_slices(data)

#data2 = tf.convert_to_tensor(data)


#data.dtypes

#user_feature_names = ['userID', 'placeID']
#user_features = data[user_feature_names]

#resto_feature_names = ['placeID']
#resto_features = data[resto_feature_names]

#user_feature_tensor = tf.convert_to_tensor(user_features)
#resto_feature_tensor = tf.convert_to_tensor(resto_features)

user_place = data2.map(lambda x: {
    "placeID": x[1],
    "userID": x[0]
})
place = data2.map(lambda x: x[1])

# Select the basic features.
#ratings = ratings.map(lambda x: {
#    "movie_title": x["movie_title"],
#    "user_id": x["user_id"]
#})
#movies = movies.map(lambda x: x["movie_title"])

#ratings

Build vocabularies to convert user ids and movie titles into integer indices for embedding layers:

In [4]:
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(user_place.map(lambda x: x["userID"]))

resto_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
resto_titles_vocabulary.adapt(place)

### Define a model

We can define a TFRS model by inheriting from `tfrs.Model` and implementing the `compute_loss` method:

In [5]:
class MovieLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.resto_model = resto_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

  #  user_embedding = self.user_model(features[0])
  #  resto_embeddings = self.resto_model(features[1])
    
    user_embedding = self.user_model(features["userID"])
    resto_embeddings = self.resto_model(features["placeID"])

    return self.task(user_embedding, resto_embeddings)

Define the two models and the retrieval task.

In [6]:
# Define user and movie models.
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocabulary_size(), 64)
])
resto_model = tf.keras.Sequential([
    resto_titles_vocabulary,
    tf.keras.layers.Embedding(resto_titles_vocabulary.vocabulary_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    place.batch(128).map(resto_model)
  )
)


### Fit and evaluate it.

Create the model, train it, and generate predictions:



In [7]:
# Create a retrieval model.
model = MovieLensModel(user_model, resto_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(user_place.batch(4096), epochs=3)

# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    place.batch(100).map(lambda title: (title, model.resto_model(title))))

# Get some recommendations.
_, titles = index(np.array(["U1067"]))
print(f"Top 3 recommendations for user U1067: {titles[0, :10]}")

Epoch 1/3
1/1 [==============================] - 1s 1s/step - factorized_top_k/top_1_categorical_accuracy: 0.0043 - factorized_top_k/top_5_categorical_accuracy: 0.0052 - factorized_top_k/top_10_categorical_accuracy: 0.0103 - factorized_top_k/top_50_categorical_accuracy: 0.0500 - factorized_top_k/top_100_categorical_accuracy: 0.0887 - loss: 8192.9766 - regularization_loss: 0.0000e+00 - total_loss: 8192.9766
Epoch 2/3
1/1 [==============================] - 0s 107ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 8034.5942 - regularization_loss: 0.0000e+00 - total_loss: 8034.5942
Epoch 3/3
1/1 [==============================] - 0s 107ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical